In [5]:
from bs4 import BeautifulSoup
import urllib.request as req
import numpy as np
from time import sleep 


####################
####  Settings  ####
####################
GENAEARION = 'sm'
SLEEP_INTERVAL = 1
TOTAL_PK_NUM = 800
OUTPUT_JSON = '.\\output\\pokemon_skills.json'


####################
####   Methods  ####
####################

# Request URL & get html
def get_soup(url):

    try:
        # Not Local 
        res = req.urlopen(url)
        soup = BeautifulSoup(res,'html.parser')
        return soup

    except urllib.error.HTTPError as http_e:
        print( http_e.code , http_e.reason, url)
        log_str = str( http_e.code) + http_e.reason +  url
        write_log(LOG_F, str(http_e.code)  )
    except Exception as e:
        print(e)

# scrapers from pk page
def scrape_pk_type(soup):
    types = []
    ul_soup = soup.find('ul',class_='type')
    for li_soup in ul_soup.findAll('img'):
        type = li_soup['alt'] 
        types.append(type)
    return types

def scrape_pk_skills(soup):
    skills = []
    
    for tr_soup in soup.findAll('tr','li',class_='move_main_row'):
        
        # Only learned by Lv UP!!
        if '技' in tr_soup.text:
            break
        
        # Learning Level
        #for td_soup in tr_soup.findAll('td',class_='move_condition_cell'):
        #    print(td_soup.text)
        
        # Skill Name
        for td_soup in tr_soup.findAll('td',class_='move_name_cell'):
            
            skill = td_soup.find('a').text
            #print(skill)
            skills.append(skill)
    
    return skills

def scrape_pk_name(soup):
    name = soup.find('div',class_='table layout_left').find('table').find('tr').find('th').text
    return name
 
def scrape_pk_attri_iter(num,soup):
    name = scrape_pk_name(soup)
    types = scrape_pk_type(soup)
    skills = scrape_pk_skills(soup)
    
    # Setting attri to json
    attri = {}
    attri['No'] = num
    attri['name'] = name
    attri['types'] = types
    attri['skills'] = skills
    return attri


####################
####   EXECUTE  ####
####################
    
if __name__ == '__main__':
    
    pk_attri ={}
    #print("STAART CRAWLING")
    for i in  range(1,TOTAL_PK_NUM+1):
        if i % 25 ==0:
            print("Current Pokemon Num = %s" % i)
        url = 'https://yakkun.com/%s/zukan/n%s' % (GENAEARION, i)
        soup = get_soup(url)
        pk_attri[i] = scrape_pk_attri_iter(i,soup)
         
        sleep(SLEEP_INTERVAL)


    import json
    fw = open(OUTPUT_JSON,'w')
    json.dump(pk_attri,fw,ensure_ascii=False,indent=2)
    fw.close()

In [20]:
# Confirm Result 
pk_attri[20]



{'No': 20,
 'name': 'ラッタ',
 'skills': ['つるぎのまい',
  'たいあたり',
  'しっぽをふる',
  'でんこうせっか',
  'きあいだめ',
  'こわいかお',
  'でんこうせっか',
  'きあいだめ',
  'かみつく',
  'おいうち',
  'ひっさつまえば',
  'ダメおし',
  'かみくだく',
  'ふいうち',
  'いかりのまえば',
  'すてみタックル',
  'がむしゃら'],
 'types': ['ノーマル']}